In [1]:
%pip install transformers torch torchtext scikit-learn pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.4 MB/s eta 0:00:00


In [2]:
%pip install torch==2.0.1 torchtext==0.15.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [3]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW
from tqdm import tqdm


data = [
    {"crimeaditionalinfo": "I have continued receiving random calls and abusive messages on WhatsApp. Someone added my number to an unknown Facebook group called 'Only Girls' and I am still getting calls from unknown numbers. Please help.", "category": "Harassment"},
    {"crimeaditionalinfo": "The fraudster is continuously messaging me, asking me to pay him money or he will send fake nude photos of me to my contacts through WhatsApp.", "category": "Fraud"},
    {"crimeaditionalinfo": "He is pretending to be a police officer and demanding money with legal-sounding threats in text messages.", "category": "Harassment"},
    {"crimeaditionalinfo": "I applied for a telecalling job and paid security fees, but the job didn’t match the description. Please help recover my financial loss.", "category": "Scam"},
    {"crimeaditionalinfo": "Received a call from someone asking for OTP verification, claiming to be from my bank. I did not provide it, but I suspect it was an attempt to steal my information.", "category": "Phishing"},
    {"crimeaditionalinfo": "An app on the Play Store accessed my contacts without permission and is now harassing me to repay a loan I never took.", "category": "Harassment"},
    {"crimeaditionalinfo": "Received a message from someone pretending to be a friend asking for an urgent loan. Later found out it wasn’t my friend.", "category": "Identity Theft"},
    {"crimeaditionalinfo": "Someone created a fake social media profile in my name and has been messaging people with offensive content.", "category": "Identity Theft"},
    {"crimeaditionalinfo": "A company promised an investment opportunity with high returns, but after I invested, they stopped responding.", "category": "Fraud"},
    {"crimeaditionalinfo": "Received multiple calls asking me to update KYC for my bank account, requesting personal details. It seemed suspicious.", "category": "Phishing"},
    {"crimeaditionalinfo": "A website claimed to sell electronics at a discount, but after placing an order, there was no response from them.", "category": "Scam"},
    {"crimeaditionalinfo": "Received an email claiming I won a prize and asking for payment to cover taxes. I suspect it's a scam.", "category": "Scam"},
    {"crimeaditionalinfo": "Someone called, pretending to be a government official and threatened legal action unless I paid a fee.", "category": "Fraud"},
    {"crimeaditionalinfo": "An unknown person has been sending harassing messages and calling my phone, affecting my personal life.", "category": "Harassment"},
    {"crimeaditionalinfo": "I was contacted by someone claiming to represent my bank, asking for my account details to prevent account suspension.", "category": "Phishing"},
    {"crimeaditionalinfo": "A man I met online asked for emergency money. He claimed to be in a difficult situation, but I believe it was a scam.", "category": "Scam"},
    {"crimeaditionalinfo": "Someone used my identity to apply for a loan and is now harassing my contacts for repayment.", "category": "Identity Theft"},
    {"crimeaditionalinfo": "Received a call from a 'representative' of a tech company, asking for remote access to my computer to fix an issue.", "category": "Phishing"},
    {"crimeaditionalinfo": "I received multiple calls and messages threatening me to repay a loan I never took. Please help.", "category": "Harassment"},
    {"crimeaditionalinfo": "Someone used my profile picture and created a fake account to impersonate me on social media.", "category": "Identity Theft"},
    {"crimeaditionalinfo": "An online store claimed to have a limited-time offer, but after paying, I didn’t receive the product.", "category": "Scam"},
    {"crimeaditionalinfo": "A person contacted me through LinkedIn offering a job but asked for upfront fees, which seemed suspicious.", "category": "Scam"},
    {"crimeaditionalinfo": "Received a suspicious email with a link to claim a refund from my bank. They requested my account and password details.", "category": "Phishing"},
    {"crimeaditionalinfo": "A man is pretending to be my relative and is calling my office, asking for personal information. This is impacting my work.", "category": "Harassment"},
    {"crimeaditionalinfo": "Received a fake message from someone posing as my friend, asking for money for an emergency.", "category": "Identity Theft"}
]




df = pd.DataFrame(data)

# Define CustomTextDataset class
class CustomTextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length, text_column='crimeaditionalinfo', label_column='category'):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

        # Encode labels
        self.label_encoder = LabelEncoder()
        self.data['encoded_label'] = self.label_encoder.fit_transform(self.data[label_column])
        self.text_column = text_column
        self.label_column = 'encoded_label'

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        text = self.data.iloc[idx][self.text_column]
        label = self.data.iloc[idx][self.label_column]

        # Tokenize the text
        tokens = self.tokenizer(
            text,
            padding='max_length',
            max_length=self.max_length,
            truncation=True,
            return_tensors="pt"
        )
        tokens = {key: val.squeeze(0) for key, val in tokens.items()}
        return tokens, torch.tensor(label)


def predict(model, tokenizer, texts, max_length=128):
    model.eval()
    predictions = []
    with torch.no_grad():
        for text in texts:
            tokens = tokenizer(
                text,
                padding='max_length',
                max_length=max_length,
                truncation=True,
                return_tensors="pt"
            )
            tokens = {key: val.to(model.device) for key, val in tokens.items()}
            output = model(**tokens)
            _, predicted_class = torch.max(output.logits, dim=1)
            predictions.append(predicted_class.item())
    return predictions


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

unique_labels = df['category'].nunique()
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=unique_labels)

MAX_LENGTH = 128
BATCH_SIZE = 16
EPOCHS = 10


train_dataset = CustomTextDataset(df, tokenizer, MAX_LENGTH, text_column='crimeaditionalinfo', label_column='category')
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


optimizer = AdamW(model.parameters(), lr=4e-5)


model.train()
for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    total_loss = 0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        tokens, labels = batch
        outputs = model(**tokens, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    print(f"Average Loss: {average_loss:.4f}")


model.save_pretrained("trained_bert_model")
tokenizer.save_pretrained("trained_bert_tokenizer")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

100%|██████████| 2/2 [00:38<00:00, 19.27s/it]


Average Loss: 0.8461
Epoch 10/10


100%|██████████| 2/2 [00:33<00:00, 16.79s/it]


Average Loss: 0.7909


('trained_bert_tokenizer/tokenizer_config.json',
 'trained_bert_tokenizer/special_tokens_map.json',
 'trained_bert_tokenizer/vocab.txt',
 'trained_bert_tokenizer/added_tokens.json')

In [4]:

unknown_texts = [
    "someone posing as my friend, asking for money for an emergency. "
]
predicted_classes = predict(model, tokenizer, unknown_texts)


for text, prediction in zip(unknown_texts, predicted_classes):
    print(f"Text: {text} | Predicted Class: {prediction} | Class Label: {train_dataset.label_encoder.inverse_transform([prediction])[0]}")

Text: someone posing as my friend, asking for money for an emergency.  | Predicted Class: 2 | Class Label: Identity Theft
